In [ ]:
import numpy as np # линейная алгебра
import pandas as pd # обработка данных
from sklearn import feature_extraction, linear_model, model_selection, preprocessing, naive_bayes, svm
from sklearn.feature_extraction.text import TfidfVectorizer ,TfidfTransformer
from sklearn.metrics import accuracy_score
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_selection import SelectPercentile, f_classif

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Препроцессинг данных для тренировочного сета
train_data = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_data = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
# Перенесем текст
corpus = []
for values in train_data['text']:
    corpus.append(values)
    
# Определим так называемые "стоп-слова"    
stop_words = stopwords.words('english') + list(punctuation)

# Используем IDF для определения редких значений с игнорированием "стоп-слов"
vectorizer = TfidfVectorizer(use_idf=True, stop_words=stop_words,smooth_idf=True)
# Получаем матрицы по тренировочным сетам
train = vectorizer.fit_transform(corpus)
test = vectorizer.transform(test_data['text'])

In [ ]:
# Используем Ridge Regression
clf = linear_model.RidgeClassifier()
# Получим значения при помощи кросс-валидации
scores = model_selection.cross_val_score(clf, train, train_data["target"], cv=3, scoring="f1")

scores
# Используем данные в модели
clf.fit(train, train_data["target"])

In [ ]:
# Прочитаем файл с примером
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
# Сделаем предикт
sample_submission["target"] = clf.predict(test)
sample_submission.head()
sample_submission.to_csv("submission.csv", index=False)